<table style="width:100%;">
    <tr style="display:none">
        <td></td>
        <td></td>
    </tr>
    <tr style="height:3em">
        <td style="width:60%;font: bold 2.5em 'Fira Sans', serif;text-align:center">SCRIPT DE REMPLISSAGE DE BASE   - ENERGIES <br>
        Durée d'exécution :  80 sec<br>
        <br>
    </tr>
</table>

<center>

pour executer le code il faut tout simplement faire  **"run all"**

## Il est important d'avoir les librairies suivantes : 
- **Pandas** afin de traiter le fichier csv
- **Pyodbc** afin de faire du sql et surtout pouvoir se connecter à la base 
- **Pycountry** convert pour avoir le nom du continent d'un pays
- **Time** d'avoir le temps d'exécution

In [ ]:
import pandas as pd
import pyodbc
import time
import pycountry_convert as pc

## Détails de connexion


In [ ]:
server = 'INFO-MSSQL-ETD'
database = 'SAE_TEAM55'
username = 'etd05'
password = 'fqnsbtc4'

## Chemin du fichier csv


In [ ]:
filename = 'Z:\\Documents\\saeTruc\\csv\\share-elec-by-source.csv'

Ceci est un tableau qui contient les continents et certains indices, il permet par la suite de traiter le cas ou le nom d'un pays est le nom d'un continent

In [ ]:
cont = ['North America','South America','Asia','Australia','Africa','Europe','World','Caribbean',
        'demographic','Euro','conflict','poor','income','IBRD','IDA','countries','OECD','small','Small',
        'Bahamas','Channel','Dem','Rep','Curacao','Hong']

## Dictionnaire des activités
ce dictionnaire va recenser les différents types d'énergie présents dans le csv en tant que clé. Et en tant que valeur tout simplement la conversion en simple mot.

In [ ]:
activities = {
    'Coal - % electricity': 'Coal',
    'Gas - % electricity': 'Gas',
    'Hydro - % electricity': 'Hydro',
    'Solar - % electricity': 'Solar',
    'Wind - % electricity': 'Wind',
    'Oil - % electricity': 'Oil',
    'Nuclear - % electricity': 'Nuclear',
    'Other renewables excluding bioenergy - % electricity': 'Other renewables excluding bioenergy',
    'Bioenergy - % electricity': 'Bioenergy'
}

La méthode `get_region_and_country(filename)` va venir regarder si le nom d'un pays est un continent afin d'avoir en tant que nom de pays le continent et le nom de continent le continent. exemple :
- AFRICA  ======> country_name = AFRICA & region_name=AFRICA


In [ ]:
def get_region_and_country(filename):
    region_name = filename
    country_name = filename
    if region_name == "Cote d'Ivoire":
        country_name = 'Ivory Coast'
    if region_name == 'Gambia, The':
        country_name = 'Gambia'
    for continent in cont:
        if continent in region_name:
            return region_name, country_name
    else:
        return region_name, country_name
    

cette courte méthode permet de vérifier que la valeur n'est pas nulle ou bien égale à 0 et également prendre que les années supérieures à 1990

In [ ]:
def should_process_row(value, year):
    """Vérifie si une ligne doit être traitée."""
    return pd.notna(value) and value != 0 and year > 1990

## Trouver une date existante ou bien la crée
`get_or_create_date_id(cursor, year)`permet d'avoir une date existante dans la base de données. Si elle n'existe pas elle va l'insérer. Elle retourne l'id de la date récupérer ou crée.

In [ ]:
def get_or_create_date_id(cursor, year):
    date_str = f"{year}-01-01"
    cursor.execute("SELECT DAT_ID FROM T_DATE_DAT WHERE DAT_DATE = ?", date_str)
    row = cursor.fetchone()
    if row:
        return row[0]
    else:
        cursor.execute("INSERT INTO T_DATE_DAT (DAT_DATE) VALUES (?)", date_str)
        cursor.execute("SELECT DAT_ID FROM T_DATE_DAT WHERE DAT_DATE = ?", date_str)
        return cursor.fetchone()[0]

## Système d'optimisation
Le but d'un chunk est de scinder les données afin d'optimiser leur traitement. Ce processus permet d'éviter de charger tout le csv d'un coup dans la mémoire. Cela optimise grandement le temps d'exécution.
`process_chunk(df_chunk, conn, batch_size=100):` permet de créer un chunk qui va faire une taille par défaut de 100 lignes. Et vérifier que la ligne est valide 


In [ ]:
def process_chunk(df_chunk, conn, batch_size=100):
    cursor = conn.cursor()
    batch = []
    start_chunk_time = time.time()

    for index, row in df_chunk.iterrows():
        year = row['Year']
        region = row['Entity']
        for column, activity in activities.items():
            value = row[column]
            if should_process_row(value, year):
                try:
                    value = round(float(value), 6)
                except ValueError:
                    print(f"Valeur invalide pour '{column}' à la ligne {index}: {value}")
                    continue

                unit = '%'

                # Ajouter les données au batch
                batch.append((region, year, activity, value, unit))

                if len(batch) >= batch_size:
                    # Insérer le batch
                    insert_batch(batch, cursor)
                    batch = []

    # Insérer le dernier batch restant
    if batch:
        insert_batch(batch, cursor)

    conn.commit()
    end_chunk_time = time.time()
    print(f"Temps de traitement du chunk: {end_chunk_time - start_chunk_time:.2f} secondes")


## Insertion des données dans la base de donnée
`insert_batch(batch, cursor)` est le coeur du script il va procéder à l'insertion des données :
- **La région**
- **Le pays**
- **Le type d'énergie**
- **Obtention de la date**
- **Insertion de la donnée**


In [ ]:
def insert_batch(batch, cursor):
    
    for data in batch:
        region, year, activity, value, unit = data

        try:
            region, country_name = get_region_and_country(region)
            # Insertion de la région
            if region:
                cursor.execute("""
                    IF NOT EXISTS (SELECT 1 FROM T_REGION_REG WHERE REG_NOM = ?)
                    BEGIN
                        INSERT INTO T_REGION_REG (REG_NOM) VALUES (?)
                    END
                """, region, region)
                cursor.execute("SELECT REG_ID FROM T_REGION_REG WHERE REG_NOM = ?", region)
                region_id = cursor.fetchone()[0]
            else:
                region_id = None

            # Insertion du  pays
            if country_name:
                cursor.execute("""
                    IF NOT EXISTS (SELECT 1 FROM T_PAYS_PYS WHERE PYS_NOM = ?)
                    BEGIN
                        INSERT INTO T_PAYS_PYS (REG_ID, PYS_NOM) VALUES (?, ?)
                    END
                """, country_name, region_id, country_name)
                cursor.execute("SELECT PYS_ID FROM T_PAYS_PYS WHERE PYS_NOM = ?", country_name)
                country_id = cursor.fetchone()[0]
            else:
                country_id = None


            
            # Insertion des energies
            cursor.execute("""
                IF NOT EXISTS (SELECT 1 FROM T_ENERGY_EGY WHERE EGY_NOM = ?)
                BEGIN
                    INSERT INTO T_ENERGY_EGY (EGY_NOM) VALUES (?)
                END
            """, (activity, activity))
            cursor.execute("SELECT EGY_ID FROM T_ENERGY_EGY WHERE EGY_NOM = ?", (activity,))
            EGY_ID = cursor.fetchone()[0]

            # Obtenir ou créer DAT_ID
            date_id = get_or_create_date_id(cursor, year)

            # Insérer les données
            cursor.execute("""
                INSERT INTO T_DATA_DTA (PYS_ID, DAT_ID, EGY_ID, DTA_VALEUR, DTA_NOM, DTA_UNITE)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (country_id, date_id, EGY_ID, value, activity, unit))
        except pyodbc.Error as e:
            print(f"Erreur lors de l'insertion des données : {e}")


## Parcour du fichier csv

In [ ]:
def process_file(filename, conn, chunksize=1000):
    total_chunks = 0
    try:
        for chunk in pd.read_csv(filename, chunksize=chunksize):
            process_chunk(chunk, conn)
            total_chunks += 1
    except pd.errors.EmptyDataError:
        print("Le fichier est vide.")
    except pd.errors.ParserError as e:
        print(f"Erreur de lecture du fichier CSV : {e}")

    print(f"Nombre total de chunks traités: {total_chunks}")

## Phase de connection et de démarrage
C'est un peu comme le main, on va essayer de se connecter à la base de données grâce à toutes les informations saisies. Et démarrer la méthode de parcours du csv.

In [ ]:
try:
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password}'
    )
    print("Connexion réussie à la base de données")
except pyodbc.Error as e:
    print(f"Erreur de connexion : {e}")
    exit()

# Mesure du temps de traitement
start_time = time.time()

# Traitement des fichiers
process_file(filename, conn)

end_time = time.time()

print('finished')
print(f"Temps total utilisé : {end_time - start_time:.2f} secondes")
conn.close()